## My Wallet App Simulator
- Execute each cell with Crtl+Enter
- Two cells will ask for an input. Make sure the respective QR (init, DCC) is visible on scree next to this browser window


Install libraries if you run this notebook for the first time

In [1]:
!pip install pyjwt cryptography protobuf pycryptodome pyjwt ecdsa


     |████████████████████████████████| 3.6 MB 5.2 MB/s 
     |████████████████████████████████| 2.0 MB 22.8 MB/s 
     |████████████████████████████████| 119 kB 58.0 MB/s 


### Scan a QR code

Create a process in the booking portal. Match the personal data (firstname, lastname) with a valid DCC that you have. 

Grab the init string from url to wallet, and decode the string at base64/decode. copy the result and past in input-box

In [18]:
import requests
import json
import jwt
import base64

invite = input('Paste Invitation string contents here: ')

base64_message = invite
base64_bytes = base64_message.encode('ascii')
message_bytes = base64.b64decode(base64_bytes)
qr_code = message_bytes.decode('ascii')

qr_code_data =  json.loads(qr_code)
print(f"QR-Code data: {json.dumps(qr_code_data, indent=4)}")

token_info = jwt.decode(qr_code_data['token'], options={"verify_signature":False})
print(f"Invitation-token: {token_info}")


Paste Invitation string contents here: eyJwcm90b2NvbCI6IkRDQ1ZBTElEQVRJT04iLCJwcm90b2NvbFZlcnNpb24iOiIxLjAuMCIsInNlcnZpY2VJZGVudGl0eSI6Imh0dHBzOi8vcGluZ2dnLm15d2lyZS5vcmcvZGlyZWN0b3J5L29mZmVyIiwicHJpdmFjeVVybCI6Imh0dHBzOi8vdmFsaWRhdGlvbi1kZWNvcmF0b3IuZXhhbXBsZSIsInRva2VuIjoiZXlKMGVYQWlPaUpLVjFRaUxDSnJhV1FpT2lKaVV6aEVNaTlYZWpWMFdUMGlMQ0poYkdjaU9pSkZVekkxTmlKOS5leUpwYzNNaU9pSm9kSFJ3Y3pvdkwyUm5ZMkV0WW05dmEybHVaeTFrWlcxdkxXVjFMWFJsYzNRdVkyWmhjSEJ6TG1WMU1UQXVhR0Z1WVM1dmJtUmxiV0Z1WkM1amIyMHZZWEJwTDJsa1pXNTBhWFI1SWl3aVpYaHdJam94TmpRMk1qSTBNREF4TENKemRXSWlPaUpoTXpkalptTXdNUzA0TUdVeExUUTNOMlV0WWpBNU9TMDRObUkwWmpGaU0yRXhZMllpZlEuZi1mYm41MGEwblAtcUJNdGZabzFFUXMzVEZLclE4elRDQ1lqOUFDMWNCa1dGQ09NM2tleTIyaVhFS2FTWm9GSjFiWThEREFWeEQ1TDF0SW8yd3R6UWciLCJjb25zZW50IjoiUGxlYXNlIGNvbmZpcm0gdG8gc3RhcnQgdGhlIERDQyBFeGNoYW5nZSBmbG93LiBJZiB5b3Ugbm90IGNvbmZpcm0sIHRoZSBmbG93IGlzIGFib3J0ZWQuIiwic3ViamVjdCI6ImEzN2NmYzAxLTgwZTEtNDc3ZS1iMDk5LTg2YjRmMWIzYTFjZiIsInNlcnZpY2VQcm92aWRlciI6IlNtaWxpbmcgZnJvbSBhYm92ZSJ9
QR-C

### Init QR code handling by wallet app
The wallet app processes the invitation:


1.   The wallet gets airline identity document to learn airline endpoints and validation service location
2.   The wallet determines location of validation service ((from airline identity document)). A choice is presented in case multiple validation services are offered.

In [30]:
import ecdsa
import base64
from Crypto.Hash import SHA256
from ecdsa.curves import NIST256p

# Load the airline identity document 
serviceIdentity = requests.get( qr_code_data['serviceIdentity'] ).json()

# Get the information from the identity document. Identity contents
## services, i.e. airline endpoint to get validation access token and to return confirmation token, and (list of) providers that offer validation
## verification methods, public keys to validate signatures or for use in encryption 

validationlist=[];
for service in serviceIdentity['service']:
    
    # This should not always be 'ValidationService-1' but the current service
    # depending on rules and requirements
    if service['id'].find('#ValidationService')>0: 
        validationlist.append(service);
        validation_service_id = service['id']        
        print(f"Validation ServiceID {validation_service_id}") 
#print(validationlist)
if len(validationlist) == 1:
  choice = 0
else:
  choice = int(input('Please enter number of service you have chosen (1,2,3): ').strip())  # This picks up the validation service
  choice=choice-1
#Make record of validation service endpoint for future use
validation_service_id=validationlist[choice]['id']
validation_service_endpoint = validationlist[choice]['serviceEndpoint'] 
print(validation_service_endpoint) 

Validation ServiceID https://dgca-booking-demo-backend-eu-test.cfapps.eu10.hana.ondemand.com/identity/service/ValidationService#ValidationService-1
Validation ServiceID https://dgca-booking-demo-backend-eu-test.cfapps.eu10.hana.ondemand.com/identity/service/ValidationService#ValidationService-2
Validation ServiceID https://dgca-booking-demo-backend-eu-test.cfapps.eu10.hana.ondemand.com/identity/service/ValidationService#ValidationService-3
Please enter number of service you have chosen (1,2,3): 1
https://dgca-validation-service-eu-test.cfapps.eu10.hana.ondemand.com


#Wallet reads the airline identity document
to get validation access token (which packs travel data and some instructions for type of validation that is expected)

The request is authenticated with invitation-token (so links back to booking and passenger)

The wallet advises its one-time identity, not sure why??

The airline hands back validation access token and a nonce (nonce, why??)

In [28]:
import ecdsa
import base64
from Crypto.Hash import SHA256
from ecdsa.curves import NIST256p

#create one-time wallet identity(key-pair) to sign the DCC on upload
userkey = ecdsa.SigningKey.generate(curve=NIST256p,hashfunc=SHA256.new) 

# App selects the AccessTokenService in Airline Identity document and requests for Validation Access token. 
for service in serviceIdentity['service']:
    if service['id'].endswith('#AccessTokenService-1'):
        response = requests.post( service['serviceEndpoint'], 
                    headers={'Authorization': f'Bearer {qr_code_data["token"]}', #initR-token goes here
                             'Content-Type':'application/json', 
                             'X-Version' : '1.0'},
                    json = {"pubKey": userkey.get_verifying_key().to_pem().decode(), 
                                    #advise pub key to validation service for inspection of signature on DCC
                            "service":validation_service_id}
                                     #advise chosen validation service
        )
 #       print ("Undecoded Response", response.status_code, response.text, '\n\n')
        
        # This is the access token for the validation service
        validator_token = response.text 
        validation_nonce = response.headers['x-nonce']
        
        token_info = jwt.decode(validator_token, options={"verify_signature":False})
        print(f"Validator_Access_Token: {json.dumps(token_info, indent=4)}")




Validator_Access_Token: {
    "iss": "https://dgca-booking-demo-eu-test.cfapps.eu10.hana.ondemand.com/api/identity",
    "exp": 1646224440,
    "sub": "a37cfc01-80e1-477e-b099-86b4f1b3a1cf",
    "aud": "https://dgca-validation-service-eu-test.cfapps.eu10.hana.ondemand.com/validate/a37cfc01-80e1-477e-b099-86b4f1b3a1cf",
    "t": 2,
    "v": "1.0",
    "iat": 1646220840,
    "vc": {
        "lang": "en-en",
        "fnt": "FOLKERTSMA",
        "gnt": "EELCO",
        "dob": "1960-01-01",
        "coa": "MK",
        "cod": "LA",
        "roa": "MK",
        "rod": "LA",
        "type": [
            "r",
            "v",
            "t"
        ],
        "category": [
            "Standard"
        ],
        "validationClock": "2022-03-03T19:50:40+00:00",
        "validFrom": "2022-03-03T11:26:40+00:00",
        "validTo": "2022-03-03T19:50:40+00:00"
    },
    "jti": "62d93d6d-e458-40ca-a870-71cd7fd84aae"
}


### Get the public key from the validation service
With the validation service endpoint that has been extracted from the decorator's identity document, we now load the validation service's identity document too to obtain its public key for encryption. 

In [29]:
validation_identity = requests.get(validation_service_endpoint+'/identity').json()

for verificationMethod in validation_identity['verificationMethod']:
    if verificationMethod['id'].endswith('ValidationServiceEncKey-1'):
        validation_service_publickey =  verificationMethod['publicKeyJwk']
        print("Selected Public Key:", validation_service_publickey )

Selected Public Key: {'x5c': ['MIIDwDCCAiigAwIBAgIEYSdvPDANBgkqhkiG9w0BAQsFADAiMSAwHgYDVQQDDBdWYWxpZGF0aW9uU2VydmljZUVuY0tleTAeFw0yMTA4MjYxMDM4NTJaFw0yMjA4MjYxMDM4NTJaMCIxIDAeBgNVBAMMF1ZhbGlkYXRpb25TZXJ2aWNlRW5jS2V5MIIBojANBgkqhkiG9w0BAQEFAAOCAY8AMIIBigKCAYEA46E7KT/egBv6lGJjA0ARXrBxGjdaw/L2Br1IyCXoeVjyqs00UgSuz5jbfOntcbvQbn+QFTePUSUFEUOmm/ngYfdJRseYydN5lk3EjaA9icZYun3fwskoJ9kqFRA7brauhH9M75sPeEjroZAGB+HJ+bho3MGZZWNbR/9EK+8Ob1piBK4eOFlACJCYHK8d/1XeGCzZ3Iiia79eRAc80sDB1rOyf3BWFBlvdbkf3u4613pAd6YimpUBLmj69iQpcBRF3aMFjLxxIyexx0Bb9mKKSo1u5jK+ILseAn44hRmhxQRUkrleWrvns64qfBA+yMvjUkzSRpx8/X5e8nTlgN2BcWJ6pSnrXSWeH3Vx5Vn5sPpcn70wdDJ5+xH5mkWLFTgKdVShCEIZTPq8j7D7TlpqwQG8aEKyXfKXQlbTzbHqvqBHlsFkkN4rw0uzIXS+SneFknCZZuNoTKpi+Tg6Bx8tpN/C2J6QRHn3ub6a8WjtylymDFnQDiYMXOQHhW0ZS/I9AgMBAAEwDQYJKoZIhvcNAQELBQADggGBANgBAOxgi0abqn9EDCwFELEHv6cat5Mwnw6Z97yeMyPNk4QVKPz5LmP+73XsOGHjoFLasje6BC7IErynvFo3d29LG7+OLsMOGEEvPh8G0svylJlbEpLQ8/CGupWnxH0TRhBJygn1ANpn71ENmUVg2IGtyoZYp/6/5ZeVDoFR22sKrLma8JeWcRU33WimulKo26Gl5

# Submit the health certificate
You can enter the health certificate data into an input field. 

Source for DCC: https://eu-dcc-validation.web.app/

Open the QR-code with Google lens (right click option in Chrome) and "copy text" to obtain health sertificate in string format 

Make sure that the name and date-of-birth match the data in the validation access token from above. 

##The wallet app now executes the following steps: 
- choose a random password (32 bytes)
- AES-encrypt the DCC with the password from above and the nonce that was obtained together with the access token 
- encrypt the password with the validation service's public key 
- sign the encrypted AES-encrypted data with the userkey that has been submitted to get the access token
- send JSON data to the validation service:
   - kid of validation server's public key that was used to encrypt password
   - encrypted dcc data
   - signature of encrypted dcc data
   - PKI encrypted password for dcc
   - constants: encScheme = RSAOAEPWithSHA256AESCBC, sigAlg = SHA256withECDSA
   - header: Access token that was previously obtained
- decode the response and print it
  - Private part of result is feedback on evaluation of individual busness rules
  - Public part of result is confirmaion token, which can be made availabe to airline if passenger wishes to do so



In [31]:
hcert = input('Please paste your HCert: ').strip()  # Option 1: Enter the HC1:-String into the input field
assert hcert.startswith('HC1:')

from base64 import b64decode
from random import randint
from Crypto.Cipher import PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from ecdsa.util import sigencode_der

# Use real random numbers for production instead!
password = bytearray([randint(0,255) for i in range(32)])

validatorkey = RSA.import_key(f'-----BEGIN CERTIFICATE-----\n{validation_service_publickey["x5c"]}\n-----END CERTIFICATE-----')
aesCipher = AES.new(password, AES.MODE_CBC,iv=b64decode(validation_nonce))

cipher = PKCS1_OAEP.new(validatorkey,hashAlgo=SHA256)
cryptKey = cipher.encrypt(password)
ciphertext= aesCipher.encrypt(pad(bytes(hcert,'utf-8'),AES.block_size))
signature = userkey.sign(ciphertext,hashfunc=SHA256.new,sigencode=sigencode_der)

headers = {'content-type': 'application/json', "Authorization":"Bearer " + validator_token ,"X-Version":"1.0"}

body = {"kid":validation_service_publickey["kid"],
        "dcc":base64.b64encode(ciphertext).decode(),
        "sig":base64.b64encode(signature).decode(),
        "encKey":base64.b64encode(cryptKey).decode(),
        "encScheme":"RSAOAEPWithSHA256AESCBC", 
        "sigAlg":"SHA256withECDSA"}

headers = {'content-type': 'application/json', "Authorization":"Bearer " +validator_token,"X-Version":"1.0"}
response = requests.post(token_info['aud'], data=json.dumps(body), headers=headers)
print ("Undecoded Response", response.status_code, response.text, '\n\n')
if response.ok:
    validate_result = jwt.decode(response.content, options={"verify_signature":False})
    print(f'Validate result message: {json.dumps(validate_result, indent=4)}')

Please paste your HCert: HC1:NCF%RN%TS3DH0RGPJB/IB-OM7533SR7694:T3XH8/F6R5IKP4HAKHR D2LTND8N:UC*GP-S4FT5D75W9AAABE34+V4YC5/HQ4QQHCR+9AFDOEA7IB65C94JB--C0PL:NAREDF-AOKEH-BK2L.UL/NAKEL66J9JA--B.9FL+9TKD0PLV1A9FEO-9-B9PKD6LFKC9LFNV6I*VL.UL 5I2FMF6IS79+PDGGBYPL0QIRR97I2HOAXL92L0. KOKGTM8$M8SNCXL9LM0*$K8KG+9RR$F+ F%J00N89M4*$K3$OHBW24FAL86H0CNCRK40YQDXI47T6QS47T9ZI4Q5%H0EM4GH0M:P:PI*VIUXFOH6ML5PVFNXUJRH0LH%Y2 UQ7S7TK24H94S2XOA3PQQRH*T2AYUUQ2/M1IRH$NI4L6N$Q%UG/YLN*JK*L5R1G-V/YLK1SF:PUJ8U-H:$N0K82/J2%R9L8 $FXYH%H0-Q3N%QXQM+96JUGZWVO/6YRDH9EP/U%J3+TVGUD6OV/7VES1AKD5VKWWSG:12L1SERV:BOXCQ-UK5S/:PP3BC3T:-4AYR89OMYM9GTAY0000FGW*LUIXG
Undecoded Response 200 eyJ0eXAiOiJKV1QiLCJraWQiOiJSQU0yU3R3N0VrRT0iLCJhbGciOiJFUzI1NiJ9.eyJzdWIiOiJhMzdjZmMwMS04MGUxLTQ3N2UtYjA5OS04NmI0ZjFiM2ExY2YiLCJpc3MiOiJodHRwczovL2RnY2EtdmFsaWRhdGlvbi1zZXJ2aWNlLWV1LXRlc3QuY2ZhcHBzLmV1MTAuaGFuYS5vbmRlbWFuZC5jb20iLCJpYXQiOjE2NDYyMjE5MDMsImV4cCI6MTY0NjMwODMwMywiY2F0ZWdvcnkiOlsiU3RhbmRhcmQiXSwiY29uZmlybWF0aW9uIjoiZXlKcmFXUWlPaUpTUV